# PRML Assignment -1, Team-33

In [ ]:
import torch
import numpy as np
import plotly.graph_objects as go

class PolynomialRegression:
    def __init__(self, degree, regularization=None, lambda_reg=0.0):
        self.degree = degree
        self.regularization = regularization
        self.lambda_reg = lambda_reg
        self.weights = None

    def design_matrix(self, X):
        """Compute the design matrix for polynomial features."""
        if X.ndim == 1:
            X = X[:, np.newaxis]

        X_poly = torch.hstack([X ** i for i in range(self.degree + 1)])

        return X_poly

    def fit(self, X, y):
        """Fit the model to the data."""
        X_poly = self.design_matrix(X)

        if self.regularization == 'quadratic':
            reg_matrix = self.lambda_reg * torch.eye(X_poly.shape[1])

            self.weights = torch.linalg.pinv(X_poly.T @ X_poly + reg_matrix) @ X_poly.T @ y

        else:
            self.weights = torch.linalg.pinv(X_poly) @ y

    def predict(self, X):
        """Predict using the polynomial model."""
        X_poly = self.design_matrix(X)
        return X_poly @ self.weights

    def plot_2d(self, X, y, dataset_name):
        """Plot the polynomial fit for 2D data."""

        X_range = torch.linspace(X.min(), X.max(), 100)
        y_pred = self.predict(X_range)

        fig = go.Figure()

        fig.add_trace(go.Scatter(x=X.numpy(), y=y.numpy(), mode='markers', name=f'{dataset_name}',
                                marker=dict(color='blue')))

        fig.add_trace(go.Scatter(x=X_range.numpy(), y=y_pred.detach().numpy(), mode='lines', name='Polynomial Fit',
                                line=dict(color='red')))

        fig.update_layout(title=f'Polynomial Regression (degree={self.degree}, lambda={self.lambda_reg}) on {dataset_name}',
                        xaxis_title='X',
                        yaxis_title='y')

        fig.show()

    def plot_3d(self, X, y, dataset_name):

        num_points = X.shape[0]
        x1_range = torch.linspace(X[:, 0].min(), X[:, 0].max(), num_points)
        x2_range = torch.linspace(X[:, 1].min(), X[:, 1].max(), num_points)

        x1_grid, x2_grid = torch.meshgrid(x1_range, x2_range)

        X_grid = torch.stack((x1_grid.flatten(), x2_grid.flatten()), dim=1)
        y_pred_grid = self.predict(X_grid)


        fig = go.Figure(data=[go.Surface(z=y_pred_grid.reshape(num_points, num_points).detach().numpy(),
                                        x=x1_grid.numpy(), y=x2_grid.numpy(), colorscale="Viridis")])

        fig.add_trace(go.Scatter3d(x=X[:, 0].numpy(), y=X[:, 1].numpy(), z=y.numpy(),
                                mode='markers', marker=dict(size=5, color='red')))

        fig.update_layout(title=f'Polynomial Regression (degree={self.degree}, lambda={self.lambda_reg}) on {dataset_name}',
                        scene=dict(xaxis_title='X1', yaxis_title='X2', zaxis_title='y'))
        fig.show()




## Dataset-1 Experiments

### Dataset 1a

#### With Regularization

In [ ]:
degrees = [9]
lambdas = [0.001, 0.1, 1]

def load_dataset_1(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, 0], dtype=torch.float32)
    y = torch.tensor(data[:, 1], dtype=torch.float32)
    return X, y

# Load datasets
# 1a  is 10 samples
X_train, y_train = load_dataset_1('dataset1/train_1a.csv')
X_val, y_val = load_dataset_1('dataset1/validation.csv')
X_test, y_test = load_dataset_1('dataset1/test.csv')


# =============================================================================

# Run with regularization

for degree in degrees:
    for lmbda in lambdas:

        best_model_reg = PolynomialRegression(degree=degree, regularization='quadratic', lambda_reg=lmbda)
        best_model_reg.fit(X_train, y_train)

        y_train_pred_reg = best_model_reg.predict(X_train)
        train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

        y_val_pred_reg = best_model_reg.predict(X_val)
        val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

        y_test_pred_reg = best_model_reg.predict(X_test)

        test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

        print("=====================================================================")
        print("Dataset 1a")
        print(f"Training with degree {degree}, lambda {lmbda} and Regularization ")
        print(f"Training MSE (with regularization): {train_mse_reg.item()}")
        print(f"Validation MSE (with regularization): {val_mse_reg.item()}")
        print(f"Test MSE (with regularization): {test_mse_reg.item()}")

        best_model_reg.plot_2d(X_train, y_train, dataset_name="Training Data - 1a")


Dataset 1a
Training with degree 9, lambda 0.001 and Regularization 
Training MSE (with regularization): 0.07880335301160812
Validation MSE (with regularization): 0.2911679446697235
Test MSE (with regularization): 0.8209449648857117


Dataset 1a
Training with degree 9, lambda 0.1 and Regularization 
Training MSE (with regularization): 0.0940273106098175
Validation MSE (with regularization): 0.3145447075366974
Test MSE (with regularization): 0.4392945468425751


Dataset 1a
Training with degree 9, lambda 1 and Regularization 
Training MSE (with regularization): 0.1225452795624733
Validation MSE (with regularization): 0.40340328216552734
Test MSE (with regularization): 0.3217655420303345


#### Without Regularization

In [ ]:
degrees_without_reg = [3, 6]

for degree in degrees_without_reg:

    best_model_reg = PolynomialRegression(degree=degree)
    best_model_reg.fit(X_train, y_train)

    y_train_pred_reg = best_model_reg.predict(X_train)
    train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

    y_val_pred_reg = best_model_reg.predict(X_val)
    val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

    y_test_pred_reg = best_model_reg.predict(X_test)

    test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

    print("=====================================================================")
    print("Dataset 1a")
    print(f"Training with degree {degree} and No Regularization ")
    print(f"Training MSE (without regularization): {train_mse_reg.item()}")
    print(f"Validation MSE (without regularization): {val_mse_reg.item()}")
    print(f"Test MSE (without regularization): {test_mse_reg.item()}")

    best_model_reg.plot_2d(X_train, y_train, dataset_name="Training Data - 1a")

Dataset 1a
Training with degree 3 and No Regularization 
Training MSE (without regularization): 0.08030286431312561
Validation MSE (without regularization): 0.44765377044677734
Test MSE (without regularization): 0.3345150053501129


Dataset 1a
Training with degree 6 and No Regularization 
Training MSE (without regularization): 0.06436879187822342
Validation MSE (without regularization): 772.0396728515625
Test MSE (without regularization): 1780.9365234375


### Dataset 1b

#### With Regularization

In [ ]:
degrees = [9]

In [ ]:
def load_dataset_1(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, 0], dtype=torch.float32)
    y = torch.tensor(data[:, 1], dtype=torch.float32)
    return X, y

# Load datasets
# 1a  is 10 samples
X_train, y_train = load_dataset_1('dataset1/train_1b.csv')
X_val, y_val = load_dataset_1('dataset1/validation.csv')
X_test, y_test = load_dataset_1('dataset1/test.csv')


# =============================================================================

# Run with regularization

for degree in degrees:
    for lmbda in lambdas:

        best_model_reg = PolynomialRegression(degree=degree, regularization='quadratic', lambda_reg=lmbda)
        best_model_reg.fit(X_train, y_train)

        y_train_pred_reg = best_model_reg.predict(X_train)
        train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

        y_val_pred_reg = best_model_reg.predict(X_val)
        val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

        y_test_pred_reg = best_model_reg.predict(X_test)

        test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

        print("=====================================================================")
        print("Dataset 1b")
        print(f"Training with degree {degree}, lambda {lmbda} and Regularization ")
        print(f"Training MSE (with regularization): {train_mse_reg.item()}")
        print(f"Validation MSE (with regularization): {val_mse_reg.item()}")
        print(f"Test MSE (with regularization): {test_mse_reg.item()}")

        best_model_reg.plot_2d(X_train, y_train, dataset_name="Training Data - 1b")
        # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
        # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")


Dataset 1b
Training with degree 9, lambda 0.001 and Regularization 
Training MSE (with regularization): 0.11464042961597443
Validation MSE (with regularization): 0.16937024891376495
Test MSE (with regularization): 0.26130685210227966


Dataset 1b
Training with degree 9, lambda 0.1 and Regularization 
Training MSE (with regularization): 0.11890459060668945
Validation MSE (with regularization): 0.16255642473697662
Test MSE (with regularization): 0.26353034377098083


Dataset 1b
Training with degree 9, lambda 1 and Regularization 
Training MSE (with regularization): 0.12449250370264053
Validation MSE (with regularization): 0.13641518354415894
Test MSE (with regularization): 0.2782191336154938


#### Without Regularization

In [ ]:
degrees_without_reg = [3, 6]

def load_dataset_1(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, 0], dtype=torch.float32)
    y = torch.tensor(data[:, 1], dtype=torch.float32)
    return X, y

# Load datasets
# 1a  is 10 samples
X_train, y_train = load_dataset_1('dataset1/train_1b.csv')
X_val, y_val = load_dataset_1('dataset1/validation.csv')
X_test, y_test = load_dataset_1('dataset1/test.csv')


# =============================================================================

# Run with regularization

for degree in degrees_without_reg:

    best_model_reg = PolynomialRegression(degree=degree)
    best_model_reg.fit(X_train, y_train)

    y_train_pred_reg = best_model_reg.predict(X_train)
    train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

    y_val_pred_reg = best_model_reg.predict(X_val)
    val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

    y_test_pred_reg = best_model_reg.predict(X_test)

    test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

    print("=====================================================================")
    print("Dataset 1b")
    print(f"Training with degree {degree} and without Regularization ")
    print(f"Training MSE (without regularization): {train_mse_reg.item()}")
    print(f"Validation MSE (without regularization): {val_mse_reg.item()}")
    print(f"Test MSE (without regularization): {test_mse_reg.item()}")

    best_model_reg.plot_2d(X_train, y_train, dataset_name="Training Data - 1b")
    # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
    # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")




Dataset 1b
Training with degree 3 and without Regularization 
Training MSE (without regularization): 0.12670524418354034
Validation MSE (without regularization): 0.16048817336559296
Test MSE (without regularization): 0.25831252336502075


Dataset 1b
Training with degree 6 and without Regularization 
Training MSE (without regularization): 0.11804911494255066
Validation MSE (without regularization): 0.17188510298728943
Test MSE (without regularization): 0.26293325424194336


## Dataset-2 Experiments

### Dataset 2a

#### With Regularization

In [ ]:
degrees = [6]
lambdas = [0.001, 0.1, 1]

def load_dataset_2(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, :2], dtype=torch.float32)
    y = torch.tensor(data[:, 2], dtype=torch.float32)
    return X, y

X_train, y_train = load_dataset_2('dataset2/Train-2a-25.csv')
X_val, y_val = load_dataset_2('dataset2/Val-50.csv')
X_test, y_test = load_dataset_2('dataset2/Test-50.csv')


# =============================================================================

# Run with regularization

for degree in degrees:
    for lmbda in lambdas:

        best_model_reg = PolynomialRegression(degree=degree, regularization='quadratic', lambda_reg=lmbda)
        best_model_reg.fit(X_train, y_train)

        y_train_pred_reg = best_model_reg.predict(X_train)
        train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

        y_val_pred_reg = best_model_reg.predict(X_val)
        val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

        y_test_pred_reg = best_model_reg.predict(X_test)

        test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

        print("=====================================================================")
        print("Dataset 2a")
        print(f"Training with degree {degree}, lambda {lmbda} and Regularization ")
        print(f"Training MSE (with regularization): {train_mse_reg.item()}")
        print(f"Validation MSE (with regularization): {val_mse_reg.item()}")
        print(f"Test MSE (with regularization): {test_mse_reg.item()}")

        best_model_reg.plot_3d(X_train, y_train, dataset_name="Training Data - 2a")
        # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
        # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")




Dataset 2a
Training with degree 6, lambda 0.001 and Regularization 
Training MSE (with regularization): 711613.75
Validation MSE (with regularization): 20217458.0
Test MSE (with regularization): 5892239.5


Dataset 2a
Training with degree 6, lambda 0.1 and Regularization 
Training MSE (with regularization): 715024.1875
Validation MSE (with regularization): 21535152.0
Test MSE (with regularization): 6131121.5


Dataset 2a
Training with degree 6, lambda 1 and Regularization 
Training MSE (with regularization): 735420.25
Validation MSE (with regularization): 22365312.0
Test MSE (with regularization): 6291886.0


#### Without Regularization

In [ ]:
degrees_without_reg = [2, 4]

def load_dataset_2(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, :2], dtype=torch.float32)
    y = torch.tensor(data[:, 2], dtype=torch.float32)
    return X, y

X_train, y_train = load_dataset_2('dataset2/Train-2a-25.csv')
X_val, y_val = load_dataset_2('dataset2/Val-50.csv')
X_test, y_test = load_dataset_2('dataset2/Test-50.csv')


# =============================================================================

# Run with regularization

for degree in degrees_without_reg:

    best_model_reg = PolynomialRegression(degree=degree)
    best_model_reg.fit(X_train, y_train)

    y_train_pred_reg = best_model_reg.predict(X_train)
    train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

    y_val_pred_reg = best_model_reg.predict(X_val)
    val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

    y_test_pred_reg = best_model_reg.predict(X_test)

    test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

    print("=====================================================================")
    print("Dataset 2a")
    print(f"Training with degree {degree} and without Regularization ")
    print(f"Training MSE (without regularization): {train_mse_reg.item()}")
    print(f"Validation MSE (without regularization): {val_mse_reg.item()}")
    print(f"Test MSE (without regularization): {test_mse_reg.item()}")

    best_model_reg.plot_3d(X_train, y_train, dataset_name="Training Data - 2a")
    # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
    # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")




Dataset 2a
Training with degree 2 and without Regularization 
Training MSE (without regularization): 2539443.75
Validation MSE (without regularization): 36386320.0
Test MSE (without regularization): 6350761.0


Dataset 2a
Training with degree 4 and without Regularization 
Training MSE (without regularization): 722300.625
Validation MSE (without regularization): 23517688.0
Test MSE (without regularization): 6667342.0


### Dataset 2b

#### With Regularization

In [ ]:
degrees = [6]
lambdas = [0.001, 0.1, 1]

def load_dataset_2(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, :2], dtype=torch.float32)
    y = torch.tensor(data[:, 2], dtype=torch.float32)
    return X, y

X_train, y_train = load_dataset_2('dataset2/Train-2b-100.csv')
X_val, y_val = load_dataset_2('dataset2/Val-50.csv')
X_test, y_test = load_dataset_2('dataset2/Test-50.csv')


# =============================================================================

# Run with regularization

for degree in degrees:
    for lmbda in lambdas:

        best_model_reg = PolynomialRegression(degree=degree, regularization='quadratic', lambda_reg=lmbda)
        best_model_reg.fit(X_train, y_train)

        y_train_pred_reg = best_model_reg.predict(X_train)
        train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

        y_val_pred_reg = best_model_reg.predict(X_val)
        val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

        y_test_pred_reg = best_model_reg.predict(X_test)

        test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

        print("=====================================================================")
        print("Dataset 2b")
        print(f"Training with degree {degree}, lambda {lmbda} and Regularization ")
        print(f"Training MSE (with regularization): {train_mse_reg.item()}")
        print(f"Validation MSE (with regularization): {val_mse_reg.item()}")
        print(f"Test MSE (with regularization): {test_mse_reg.item()}")

        best_model_reg.plot_3d(X_train, y_train, dataset_name="Training Data - 2b")
        # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
        # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")




Dataset 2b
Training with degree 6, lambda 0.001 and Regularization 
Training MSE (with regularization): 3155429.75
Validation MSE (with regularization): 18584380.0
Test MSE (with regularization): 3464299.25


Dataset 2b
Training with degree 6, lambda 0.1 and Regularization 
Training MSE (with regularization): 3155571.5
Validation MSE (with regularization): 18630478.0
Test MSE (with regularization): 3465108.25


Dataset 2b
Training with degree 6, lambda 1 and Regularization 
Training MSE (with regularization): 3162427.0
Validation MSE (with regularization): 18907002.0
Test MSE (with regularization): 3470084.75


#### Without Regularization

In [ ]:
degrees_without_reg = [2, 4]

def load_dataset_2(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    X = torch.tensor(data[:, :2], dtype=torch.float32)
    y = torch.tensor(data[:, 2], dtype=torch.float32)
    return X, y

X_train, y_train = load_dataset_2('dataset2/Train-2b-100.csv')
X_val, y_val = load_dataset_2('dataset2/Val-50.csv')
X_test, y_test = load_dataset_2('dataset2/Test-50.csv')


# =============================================================================

# Run without regularization

for degree in degrees_without_reg:

    best_model_reg = PolynomialRegression(degree=degree)
    best_model_reg.fit(X_train, y_train)

    y_train_pred_reg = best_model_reg.predict(X_train)
    train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

    y_val_pred_reg = best_model_reg.predict(X_val)
    val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

    y_test_pred_reg = best_model_reg.predict(X_test)

    test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

    print("=====================================================================")
    print("Dataset 2b")
    print(f"Training with degree {degree}, lambda {lmbda} without Regularization ")
    print(f"Training MSE (without regularization): {train_mse_reg.item()}")
    print(f"Validation MSE (without regularization): {val_mse_reg.item()}")
    print(f"Test MSE (without regularization): {test_mse_reg.item()}")

    best_model_reg.plot_3d(X_train, y_train, dataset_name="Training Data - 2b")
    # best_model_reg.plot_2d(X_val, y_val, dataset_name="Validation Data - 1")
    # best_model_reg.plot_2d(X_test, y_test, dataset_name="Test Data - 1")




Dataset 2b
Training with degree 2, lambda 1 without Regularization 
Training MSE (without regularization): 5373901.5
Validation MSE (without regularization): 28472468.0
Test MSE (without regularization): 5712845.0


Dataset 2b
Training with degree 4, lambda 1 without Regularization 
Training MSE (without regularization): 3310208.25
Validation MSE (without regularization): 21141170.0
Test MSE (without regularization): 3419321.5


### Dataset-3

#### With Regularization

In [ ]:
degrees = [3]
lambdas = [0.000001, 0.0001, 0.1]

def load_dataset_3(data_filename, label_filename):

    data = np.loadtxt(data_filename, delimiter=',')
    X = torch.tensor(data, dtype=torch.float32)

    y = torch.tensor(np.loadtxt(label_filename), dtype=torch.float32).reshape(-1, 1)

    return X, y


X_train, y_train = load_dataset_3(
    'dataset3/train_data.csv',
    'dataset3/train_label.csv'
)
X_val, y_val = load_dataset_3(
    'dataset3/val_data.csv',
    'dataset3/val_label.csv'
)
X_test, y_test = load_dataset_3(
    'dataset3/test_data.csv',
    'dataset3/test_label.csv'
)


# =============================================================================

# Run with regularization

for degree in degrees:
    for lmbda in lambdas:

        best_model_reg = PolynomialRegression(degree=degree, regularization='quadratic', lambda_reg=lmbda)
        best_model_reg.fit(X_train, y_train)

        y_train_pred_reg = best_model_reg.predict(X_train)
        train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

        y_val_pred_reg = best_model_reg.predict(X_val)
        val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

        y_test_pred_reg = best_model_reg.predict(X_test)

        test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

        print("=====================================================================")
        print("Dataset 3")
        print(f"Training with degree {degree}, lambda {lmbda} and Regularization ")
        print(f"Training MSE (with regularization): {train_mse_reg.item()}")
        print(f"Validation MSE (with regularization): {val_mse_reg.item()}")
        print(f"Test MSE (with regularization): {test_mse_reg.item()}")


Dataset 3
Training with degree 3, lambda 1e-06 and Regularization 
Training MSE (with regularization): 27.194032669067383
Validation MSE (with regularization): 26.80768394470215
Test MSE (with regularization): 27.114795684814453
Dataset 3
Training with degree 3, lambda 0.0001 and Regularization 
Training MSE (with regularization): 27.194032669067383
Validation MSE (with regularization): 26.80768394470215
Test MSE (with regularization): 27.114795684814453
Dataset 3
Training with degree 3, lambda 0.1 and Regularization 
Training MSE (with regularization): 27.194032669067383
Validation MSE (with regularization): 26.80768394470215
Test MSE (with regularization): 27.114795684814453


#### Without Regularization

In [ ]:
degrees = [2]

def load_dataset_3(data_filename, label_filename):

    data = np.loadtxt(data_filename, delimiter=',')
    X = torch.tensor(data, dtype=torch.float32)

    y = torch.tensor(np.loadtxt(label_filename), dtype=torch.float32).reshape(-1, 1)

    return X, y


X_train, y_train = load_dataset_3(
    'dataset3/train_data.csv',
    'dataset3/train_label.csv'
)
X_val, y_val = load_dataset_3(
    'dataset3/val_data.csv',
    'dataset3/val_label.csv'
)
X_test, y_test = load_dataset_3(
    'dataset3/test_data.csv',
    'dataset3/test_label.csv'
)


# =============================================================================

# Run without regularization

for degree in degrees:

    best_model_reg = PolynomialRegression(degree=degree)
    best_model_reg.fit(X_train, y_train)

    y_train_pred_reg = best_model_reg.predict(X_train)
    train_mse_reg = torch.mean((y_train_pred_reg - y_train) ** 2)

    y_val_pred_reg = best_model_reg.predict(X_val)
    val_mse_reg = torch.mean((y_val_pred_reg - y_val) ** 2)

    y_test_pred_reg = best_model_reg.predict(X_test)

    test_mse_reg = torch.mean((y_test_pred_reg - y_test) ** 2)

    print("=====================================================================")
    print("Dataset 3")
    print(f"Training with degree {degree} and without Regularization ")
    print(f"Training MSE (without regularization): {train_mse_reg.item()}")
    print(f"Validation MSE (without regularization): {val_mse_reg.item()}")
    print(f"Test MSE (without regularization): {test_mse_reg.item()}")


Dataset 3
Training with degree 2 and without Regularization 
Training MSE (without regularization): 5.025601863861084
Validation MSE (without regularization): 5.023364067077637
Test MSE (without regularization): 5.32115364074707
